In [1]:
""" Issue: option 1 already has its deadtime calculated, so the deadtime returned here will override the previous one """

def _select_option(max_slew_time, Now, period, incomplete, bin, prev_tile,
                   telescope_boresight, solar_array, slew_angle_data, slew_time_data, satellite):
    """
    Select the following options:
        1 : stay on the previous tile
        2 : slew to the earliest incomplete tile
        3 : slew to a new tile at the opposite end of the bin

    Returns
    -------
        option, deadtime : int, float or None
    """
    if incomplete:
        # If there are incomplete tiles
        remain_tile = incomplete[0][0]
        if bin:
            # and also new tiles in the bin
            new_tile = bin[0] if top else bin[-1]
            # First check if the slew time exceeds the 'max_slew_time' 
            slew_to_new = large_slew(Now, Now, prev_tile, new_tile, telescope_boresight, 
                                 solar_array, slew_angle_data, slew_time_data, satellite) 
            slew_to_remain = large_slew(Now, Now, prev_tile, remain_tile, telescope_boresight, 
                                 solar_array, slew_angle_data, slew_time_data, satellite)
            print('slew_to_new time:', slew_to_new, 's')
            print('slew_to_remain time:', slew_to_remain, 's')
            new_deadtime = find_deadtime(new_tile, Now, slew_to_new+settle+GSacquire, subexposure, period, satellite)
            remain_deadtime = find_deadtime(remain_tile, Now, slew_to_remain+settle+GSacquire, subexposure, period, satellite)
            if slew_to_new < max_slew_time and slew_to_remain < max_slew_time:
                # If both slews are possible, then choose whichever one is visible first
                if len(incomplete) > 1 or remain_deadtime < new_deadtime:
                    # choose 'remain_tile' if there are multiple incompleted tiles 
                    # or if the 'remain_tile' is visible first.
                    return 2, remain_deadtime
                else:
                    # or else choose the new tile
                    return 3, new_deadtime
            elif slew_to_new < max_slew_time and not slew_to_remain < max_slew_time:
                # If only the slew_to_new is possible, then select the new tile
                return 3, new_deadtime
            elif not slew_to_new < max_slew_time and slew_to_remain < max_slew_time: 
                # If only the slew_to_remain is possible, then select the incomplete tile
                return 2, remain_deadtime
            else:
                # If none of the slews are possible
                # stick with the previous tile
                return 1, None
        else:
            # and no new tiles 
            # then finish the incomplete tiles
            return 2, remain_deadtime
    else:
        # If there are no incomplete tiles
        # pick a new tile
        return 3, new_deadtime    

In [ ]:
"""
Thoughts: just don't consider visibilty in selecting option

Just differentiate between staying on previous or go to a different tile (whether this tile is incomplete or new)
"""

In [5]:
num=1
if num ==1:
    num += 1
    if num == 2:
        print('num:', num)
        continue
    print('failed')
if num == 2:
    print('hi')
    

SyntaxError: 'continue' not properly in loop (179496941.py, line 6)

In [ ]:
""" Wide Survey Parameters """
exposure = 1000 
# Define maximum slew time allowed in seconds
max_slew_time = 8*60

start_day = None
# Users can input the 'start_day'
# The default version uses the day corresponding to when the smallest RA tile in the Survey is in the anti Sun direction.
if start_day == None:
    # find the smallest tile center RA
    smallest_ra = min(bins[0], key=lambda x: x[0])[0]
    start_day = _find_anti_SunRA_day(smallest_ra, VE)
    print(start_day)

# Define the top most and bottom most tile of the first bin 
top_tile, bottom_tile = tile_target_bins[0][0], tile_target_bins[0][-1]
# their visibility windows with minute time step
top_vis = top_tile.visibility_windows(start_day, start_day+period*u.min, 60, satellite)
bot_vis = bottom_tile.visibility_windows(start_day, start_day+period*u.min, 60, satellite)

# Define starting time and tiling direction
# assume starting from the top
top = True 
# assume starting at the beginning of the day
start_time = start_day
# if only bottom is visible 
if not top_vis.includes(start_time) and bot_vis.includes(start_time):
    # start from the bottom
    top = not top 
    start_time = bot_vis.windows[0][0]
# if both are not visible
elif not top_vis.includes(start_time) and not bot_vis.includes(start_time):
    # pick the one that starts late
    if top_vis.windows[0][0].value > bot_vis.windows[0][0].value:
        start_time = top_vis.windows[0][0]
    else:
        top = not top 
        start_time = bot_vis.windows[0][0]
# if both are visible        
elif top_vis.includes(start_time) and bot_vis.includes(start_time):
    # pick the one that ends late
    if bot_vis.windows[0][1].value > top_vis.windows[0][1].value:
        top = not top
        start_time = bot_vis.windows[0][0]

Now = start_time
print('start_time:', Now)

""" Tracker """
Slew = 0
Settle = 0
GS_Acquisition = 0
Exposure = 0
Dither = 0
Readout = 0
Data_Onboard = 0

# Stores the segments of continuous deadtime in seconds
deadtime_segments = []

# Stores tile center coordinates (RA, DEC) in degrees, where RA values are in [-180°, 180°] 
tiling_sequence = []
# The number of tiles imaged or in the process of imaging
tile_num = 0
# Stores incomplete tile information
incomplete = [] # a list of tuples in the form (tile, subexposure_count)

import timeit
code_start = timeit.default_timer()

for bin in bins_copy[:1]:    
    while bin or incomplete:
        print(f'\033[1mbin length: {len(bin)}\033[0m')
        print(f'\033[1mincomplete length: {len(incomplete)}\033[0m')
        print(Now)
    
        """ Slew + Settle (including FGS scan ID) + GS Acquisition """
        if tile_num == 0:
            # Select the initial tile
            tile_num += 1
            print(f'\033[1mtop: {top}\033[0m')
            if top:
                tile = bin.pop(0)
            else:
                tile = bin.pop()
            # document the tiling sequence
            tiling_sequence.append((tile.ra, tile.dec))
            
            # Assume the satellite slews from its CVZ (continuous viewing zone) center
            CVZ = InertialTarget(*_find_CVZcenter(satellite, Now))
            slew = large_slew(Now, Now, CVZ, tile, telescope_boresight, solar_array,
                              slew_angle_data, slew_time_data, satellite)
            print('initial slew:', slew)
        else:
            # For the subsequent tiles
            # Determine which tile to observe
            option = _select_option(max_slew_time, Now, period, incomplete, bin, prev_tile,
                                    telescope_boresight, solar_array, slew_angle_data, slew_time_data, satellite)
            if option == 1:
                print('select new tile')
                # Select a new tile from the bin
                tile_num += 1
                print(f'\033[1mtop: {top}\033[0m')
                if top:
                    tile = bin.pop(0)
                else:
                    tile = bin.pop()
                subexposure_count = 0
                # document the tiling sequence
                tiling_sequence.append((tile.ra, tile.dec))
                
            if option == 2:
                print('select incomplete tile')
                # If the earliest incomplete tile is the same as the previous tile
                # go to option 3
                if incomplete[0][0] == prev_tile:
                    option = 3
                # Select the incomplete tile
                tile, subexposure_count = incomplete.pop(0)
                # document the tiling sequence
                tiling_sequence.append((tile.ra, tile.dec))
            
            if option == 3:
                print('continue previous tile')
                # Select to continue the previous tile
                tile, subexposure_count = incomplete.pop(0)
                top = not top # switch back the tiling direction
                # Update 'Now' to be after deadtime
                Now += deadtime*u.s
                Deadtime += deadtime
                print('Time Elapsed:', print_time((Now - start_time).sec))
                
            if tile == prev_tile:
                # If the subsequent tile is the same as the previous tile
                # then there's no slew
                slew = 0
            else:
                # If the subsequent tile is a different tile
                # check if there is deadtime
                estimate_slew = large_slew(prev_time, Now, prev_tile, tile, telescope_boresight, 
                                           solar_array, slew_angle_data, slew_time_data, satellite)
                print('estimated slew time:', estimate_slew)
                duration = estimate_slew + settle + GSacquire
                deadtime = find_deadtime(tile, Now, duration, subexposure, period, satellite)
                if deadtime != 0:
                    # Update 'Now' to be after deadtime
                    Now += deadtime*u.s
                    deadtime_segments.append(deadtime)
                    print('Time Elapsed:', print_time((Now - start_time).sec))
                # and calculate the slew to it
                slew = large_slew(prev_time, Now, prev_tile, tile, telescope_boresight, 
                             solar_array, slew_angle_data, slew_time_data, satellite)
                print('actual slew time:', slew)
                
        print(f'\033[1mtile_num: {tile_num}\033[0m')
        # Slew, Settle, GS Acquisition
        Slew += slew
        Settle += settle
        GS_Acquisition += GSacquire
        # Update time
        Now += slew*u.s + settle*u.s + GSacquire*u.s
        print('Time Elapsed:', print_time((Now - start_time).sec))
                
        """ Exposure + Readout """
        while subexposure_count < subexposure_num:
            # Check if there is deadtime
            deadtime = find_deadtime(tile, Now, 0, subexposure, period, satellite) 
            # When deadtime occurs, switch tiling direction
            if deadtime != 0:
                prev_tile = tile
                prev_time = Now
                incomplete.append((tile, subexposure_count))
                top = not top
                break
                
            # Imaging
            Exposure += subexposure
            Dither += dither/subexposure_num
            Readout += readout
            Data_Onboard += data
            # Update time
            Now += (subexposure + dither/subexposure_num + readout)*u.s
            print('imaging + dither per subexposure:', subexposure + dither/subexposure_num + readout, 's')
            subexposure_count += 1
        
        # Complete all subexposures
        prev_tile = tile
        prev_time = Now

    # After a bin is completed
    # switch the tiling direction
    top = not top
runtime = timeit.default_timer() - code_start
print(f"runtime: {runtime/60} min")

In [ ]:
def _select_option(min_view_eff, max_slew_time, Now, period, subexposure, incomplete, bin, prev_tile,
                   telescope_boresight, solar_array, slew_angle_data, slew_time_data, satellite):
    """
    Selects the following options
        1 : slew to a new tile 
        2 : slew to the earliest incomplete tile
        3 : stay on the previous tile
    Prioritize choosing the highest option (i.e., 1) as long as the slew to it doesn't exceed 'max_slew_time'. 
    
    Parameters
    ----------
        max_slew_time : float
            Maximum slew time allowed (in seconds) for slewing in between tiles.
    Returns
    -------
        option : int
    """
    """ assume have not switched tiling direction """
    # First, determine whether to switch tiling direction
    if bin:
        adjacent_tile = bin[0] if top else bin[-1] # the next tile in the current tiling direction
        opposite_tile = bin[0] if not top else bin[-1] # the tile at the opposite end of the bin

        # Check if the adjacent tile will be visible after slewing to it
        estimate_slew = large_slew(prev_time, Now, prev_tile, adjacent_tile, telescope_boresight, 
                                   solar_array, slew_angle_data, slew_time_data, satellite)
        duration = estimate_slew + settle + GSacquire
        deadtime = find_deadtime(adjacent_tile, Now, duration, subexposure, period, satellite)
        if deadtime != 0:
            # If the adjacent tile will not be visible after the slew
            # consider the opposite tile's visibility
            deadtime = find_deadtime(opposite_tile, Now, duration, subexposure, period, satellite)
            if (deadtime/60)/period < min_view_eff:
                # If the opposite tile is near the CVZ
                # then change the tiling direction
                print('change tiling direciton')
                top = not top
    
    # Next, check the slew length
    if incomplete:
        # If there are incomplete tiles
        remain_tile = incomplete[0][0]
        if bin:
            # and also new tiles in the bin
            new_tile = bin[0] if top else bin[-1]
            # Find the slew time to both tiles
            slew_to_remain = large_slew(Now, Now, prev_tile, remain_tile, telescope_boresight, 
                                 solar_array, slew_angle_data, slew_time_data, satellite)
            slew_to_new = large_slew(Now, Now, prev_tile, new_tile, telescope_boresight, 
                                 solar_array, slew_angle_data, slew_time_data, satellite) 
            print('slew_to_remain time:', slew_to_remain, 's')
            print('slew_to_new time:', slew_to_new, 's')
            
            if slew_to_new <= max_slew_time:
                # If 'slew_to_new' doesn't exceed the limitation, select the new tile
                # prioritize this
                return 1
            elif slew_to_remain <= max_slew_time:
                # If 'slew_to_remain' doesn't exceed the limitation, select the incomplete tile
                return 2
            else:
                # If both slews exceed 'max_slew_time', stay on the previous tile
                return 3
        else:
            # but no new tiles 
            # then finish the incomplete tiles
            return 2
    else:
        # If there are no incomplete tiles
        # pick a new tile
        return 1  

In [ ]:
""" Wide Survey Parameters """
exposure = 1000 
# Define the minimum viewing efficiency (ratio) of a tile within an orbit 
# that justifies switching the tiling direction
min_view_eff = 0.9
# Define maximum slew time allowed in seconds
max_slew_time = 3*60

start_day = None
# Users can input the 'start_day'
# The default version uses the day corresponding to when the smallest RA tile in the Survey is in the anti Sun direction.
if start_day == None:
    # find the smallest tile center RA
    smallest_ra = min(bins[0], key=lambda x: x[0])[0]
    start_day = _find_anti_SunRA_day(smallest_ra, VE)

# Define the top most and bottom most tile of the first bin 
top_tile, bottom_tile = tile_target_bins[0][0], tile_target_bins[0][-1]
# and find their respective start time (of a new visible cycle)
top_start = top_tile.visibility_windows(start_day, start_day+period*u.min, 60, satellite).get_start_time()
bottom_start = bottom_tile.visibility_windows(start_day, start_day+period*u.min, 60, satellite).get_start_time()

# Define the survey's 'start_time' and tiling direction
if top_start < bottom_start:
    top = True
    start_time = top_start
else:
    top = False
    start_time = bottom_start

Now = start_time
print('start_time:', Now)

""" Tracker """
Slew = 0
Settle = 0
GS_Acquisition = 0
Exposure = 0
Dither = 0
Readout = 0
Data_Onboard = 0

# Stores the segments of continuous deadtime in seconds
deadtime_segments = []

# Stores tile center coordinates (RA, DEC) in degrees, where RA values are in [-180°, 180°] 
tiling_sequence = []
# The number of tiles imaged or in the process of imaging
tile_num = 0
# Stores incomplete tile information
incomplete = [] # a list of tuples in the form (tile, subexposure_count)

import timeit
code_start = timeit.default_timer()

for bin in bins_copy[:1]:    
    while bin or incomplete:
        print(f'\033[1mTime Elapsed: {print_time((Now - start_time).sec)}\033[0m')
        print(f'\033[1mbin length: {len(bin)}\033[0m')
        print(f'\033[1mincomplete length: {len(incomplete)}\033[0m')
    
        """ Slew + Settle (including FGS scan ID) + GS Acquisition """
        if tile_num == 0:
            # Select the initial tile
            tile_num += 1
            print(f'\033[1mtop: {top}\033[0m')
            tile = bin.pop(0) if top else bin.pop()
            subexposure_count = 0
            print(f'\033[1mtile_num: {tile_num}\033[0m')
            # Assume the satellite slews from its CVZ (continuous viewing zone) center
            CVZ = InertialTarget(*_find_CVZcenter(satellite, Now))
            slew = large_slew(Now, Now, CVZ, tile, telescope_boresight, solar_array,
                              slew_angle_data, slew_time_data, satellite)
            print('initial slew:', print_time(slew))
            # Slew, Settle, GS Acquisition
            Slew += slew
            Settle += settle
            GS_Acquisition += GSacquire
            # Update time
            Now += slew*u.s + settle*u.s + GSacquire*u.s
            print('settle + GS acquisition:', print_time(settle+GSacquire))
            print(f'\033[1mTime Elapsed: {print_time((Now - start_time).sec)}\033[0m')
        else:
            # For the subsequent tiles, determine which tile to observe
            # First, determine whether to switch tiling direction
            if bin:
                adjacent_tile = bin[0] if top else bin[-1] # the next tile in the current tiling direction
                opposite_tile = bin[0] if not top else bin[-1] # the tile at the opposite end of the bin
        
                # Check if the adjacent tile will be visible after slewing to it
                estimate_slew = large_slew(prev_time, Now, prev_tile, adjacent_tile, telescope_boresight, 
                                           solar_array, slew_angle_data, slew_time_data, satellite)
                print('estimated slew to adjacent tile:', print_time(estimate_slew))
                duration = estimate_slew + settle + GSacquire
                print("adjacent tile's deadtime:")
                deadtime = find_deadtime(adjacent_tile, Now, duration, subexposure, period, satellite)
                if deadtime != 0:
                    # If the adjacent tile will not be visible after the slew
                    # consider the opposite tile's visibility after slewing to it
                    estimate_slew = large_slew(prev_time, Now, prev_tile, opposite_tile, telescope_boresight, 
                                               solar_array, slew_angle_data, slew_time_data, satellite)
                    print('estimated slew to opposite tile:', print_time(estimate_slew))
                    duration = estimate_slew + settle + GSacquire
                    print("opposite tile's deadtime:")
                    deadtime = find_deadtime(opposite_tile, Now, duration, subexposure, period, satellite)
                    view_eff = (period - (deadtime/60)) / period
                    print('viewing eff:', view_eff)
                    if view_eff > min_view_eff and estimate_slew <= max_slew_time:
                        # If the opposite tile is near the CVZ, i.e. has high viewing efficiency 
                        # and the slew is not too large
                        # then change the tiling direction
                        print('\033[1mchange tiling direction\033[0m')
                        top = not top
            
            # Next, check the slew length
            if incomplete:
                # If there are incomplete tiles
                remain_tile = incomplete[0][0]
                if bin:
                    # and also new tiles in the bin
                    new_tile = bin[0] if top else bin[-1]
                    # Find the slew time to both tiles
                    if remain_tile == prev_tile:
                        slew_to_remain = 0
                    else:
                        slew_to_remain = large_slew(Now, Now, prev_tile, remain_tile, telescope_boresight, 
                                         solar_array, slew_angle_data, slew_time_data, satellite)
                    slew_to_new = large_slew(Now, Now, prev_tile, new_tile, telescope_boresight, 
                                         solar_array, slew_angle_data, slew_time_data, satellite) 
                    print('slew_to_remain time:', slew_to_remain, 's')
                    print('slew_to_new time:', slew_to_new, 's')

                    # If both slews exceed 'max_slew_time', stay on the previous tile
                    if slew_to_remain > max_slew_time and slew_to_new > max_slew_time:
                        option = 3 
                    # Else choose whichever's slew is smaller
                    elif slew_to_remain < slew_to_new:
                        option = 2 # the incomplete tile
                        if remain_tile == prev_tile:
                            option = 3 # check if the incomplete tile is actualy the previous tile
                    else:
                        option = 1 # a new tile  
                else:
                    # but no new tiles in the bin
                    # then finish the incomplete tiles
                    option = 2
            else:
                # If there are no incomplete tiles
                # pick a new tile
                option = 1  
                
            if option == 1:
                print('\033[1mselect new tile\033[0m')
                # Select a new tile from the bin
                tile_num += 1
                print(f'\033[1mtop: {top}\033[0m')
                tile = bin.pop(0) if top else bin.pop()
                subexposure_count = 0
                # document the tiling sequence
                tiling_sequence.append((tile.ra, tile.dec)) 
            elif option == 2:
                print('\033[1mselect incomplete tile\033[0m')
                # Select the incomplete tile
                tile, subexposure_count = incomplete.pop(0)
                # document the tiling sequence
                tiling_sequence.append((tile.ra, tile.dec))
            elif option == 3:
                print('\033[1mcontinue previous tile\033[0m')
                # Select to continue the previous tile
                tile, subexposure_count = incomplete.pop(0)
                # Update 'Now' to be after deadtime
                Now += deadtime*u.s
                deadtime_segments.append(deadtime)
                print(f'\033[1mTime Elapsed: {print_time((Now - start_time).sec)}\033[0m')

            print(f'\033[1mtile_num: {tile_num}\033[0m')
            
            if tile != prev_tile:
                # If the subsequent tile is a different tile
                # check if there is deadtime
                estimate_slew = large_slew(prev_time, Now, prev_tile, tile, telescope_boresight, 
                                           solar_array, slew_angle_data, slew_time_data, satellite)
                duration = estimate_slew + settle + GSacquire
                deadtime = find_deadtime(tile, Now, duration, subexposure, period, satellite)
                if deadtime != 0:
                    # Update 'Now' to be after deadtime
                    Now += deadtime*u.s
                    deadtime_segments.append(deadtime)
                    print(f'\033[1mTime Elapsed: {print_time((Now - start_time).sec)}\033[0m')
                # and calculate the slew to it
                slew = large_slew(prev_time, Now, prev_tile, tile, telescope_boresight, 
                             solar_array, slew_angle_data, slew_time_data, satellite)
                print('actual slew time:', print_time(slew))
                # Slew, Settle, GS Acquisition
                Slew += slew
                Settle += settle
                GS_Acquisition += GSacquire
                # Update time
                Now += slew*u.s + settle*u.s + GSacquire*u.s
                print('settle + GS acquisition:', print_time(settle+GSacquire))
                print(f'\033[1mTime Elapsed: {print_time((Now - start_time).sec)}\033[0m')
                
        """ Exposure + Readout """
        while subexposure_count < subexposure_num:
            # Check if there is deadtime
            deadtime = find_deadtime(tile, Now, 0, subexposure, period, satellite) 
            # When deadtime occurs, stop imaging to reselect the tile
            if deadtime != 0:
                incomplete.append((tile, subexposure_count))
                break
                
            # Imaging
            Exposure += subexposure
            Dither += dither/subexposure_num
            Readout += readout
            Data_Onboard += data
            # Update time
            Now += (subexposure + dither/subexposure_num + readout)*u.s
            print('imaging + dither per subexposure:', subexposure + dither/subexposure_num + readout, 's')
            subexposure_count += 1
        
        # Document the previous tile and time
        prev_tile = tile
        prev_time = Now

runtime = timeit.default_timer() - code_start
print(f"runtime: {runtime/60} min")

In [1]:
def is_visible(tile, Now, duration, subexposure, satellite):
    """
    Determines if the 'tile' is visible throughout the 'subexposure' time, which starts 'duration' after 'Now'.

    Parameters
    ----------
        tile : Target object
        Now : Astropy Time object
            The current time of observation.
        duration : float
            The time in seconds from 'Now' before the perceived imaging start time (assuming no deadtime).
        subexposure : float
            The continuous open-shutter time in seconds.
        satellite : skyfield.sgp4lib.EarthSatellite
    Returns
    -------
        bool 
            'True' means the 'tile' is visible. 'False' means the 'tile' is occulted during the 'subexposure' time. 
    """
    # the imaging start time
    start_time = Now + duration*u.s
    time_range = np.arange(0, subexposure, 5)
    time_array = start_time + time_range*u.s
    # Checking every 5 seconds
    for time in time_array:
        if not tile.avoid_bright_objects(satellite, time):
            return False
    return True

def find_deadtime(tile, Now, duration, subexposure, satellite):
    """ 
    Calculates the time duration from 'Now' before the 'tile' becomes visible again. 
    The visibility windows over an orbital period must be defined first. 
    
    Parameters
    ----------
        tile : Target object
        Now : Astropy Time object
            The current time of observation.
        duration : float
            The time in seconds from 'Now' before the perceived imaging start time (assuming no deadtime).
        subexposure : float
            The continuous open-shutter time in seconds.
        satellite : skyfield.sgp4lib.EarthSatellite
    Returns
    -------
        deadtime : float
    """
    orbit_vis_windows = tile.vis_windows
    if orbit_vis_windows == None:
        raise TypeError("Must define the tile's visibility over an orbital period first, and store it in the 'vis_windows' attribute.")
    if orbit_vis_windows.num == 0:
        raise ValueError("The tile has no visibility windows.")
    
    # First check if deadtime is needed    
    if tile.avoid_bright_objects(satellite, Now+duration*u.s) and tile.avoid_bright_objects(satellite, Now+duration*u.s+subexposure*u.s):
        # If the tile is visible throughout the open-shutter time, no deadtime is required
        deadtime = 0
    else:
        # Deadtime is the time from 'Now' to the start of new visible cycle.
        deadtime = (orbit_vis_windows.get_start_time() - Now).sec
        print(f'deadtime = {round(deadtime/60, 2)} min')
    return deadtime

In [ ]:
import numpy as np
import astropy.units as u
from astropy.time import Time
from skyfield.api import load

""" Functions for Observation """
def _find_CVZcenter(satellite, time):
    """
    Find the equitorial coordinates (ra, dec) corresponding to the continuous viewing zone (CVZ) center 
    of the satellite at the given time.

    Parameters
    ----------
        satellite : skyfield.sgp4lib.EarthSatellite
        time : Astropy Time object
    Returns
    -------
        CVZ_ra, CVZ_dec : tuple
    """
    # Define the Right Ascension of the CVZ center
    # as the RA of anti Sun direction
    t = load.timescale().from_astropy(time)
    CVZ_ra, _ = cart_to_equi(-SolarSystemTarget('sun').pointing(satellite, time))
    # Define the Declination of the CVZ center
    inclo = np.degrees(satellite.model.inclo) # orbital inclination
    CVZ_dec = 90 - inclo
    return CVZ_ra, CVZ_dec

def _find_anti_SunRA_day(surveyRA, VE):
    """
    Finds the day when the Solar longitude (the Sun's RA position) is positioned 180° away 
    along the ecliptic from 'surveyRA' on the celestial sphere.
    i.e., the day when the survey is in the opposite direction from the Sun as seen from Earth. 
    
    Parameters
    ----------
        surveyRA : float
            Right Ascension value in [-180°, +180°] range. 
        VE : Astropy Time object
                Vernal Equinox of the current year in UTC.
                e.g., VE = Time("2024-3-20 3:6:0") for March 20, 2024 at 03:06 UTC 
    Returns
    -------
        anti_SunRA_day : Astropy Time object
    """
    # lamda: Sun ecliptic longitude; lamda = 0° on Vernal Equinox
    # anti_lamda: opposite of Sun ecliptic longitude starting on Vernal Equinox
    anti_lamda = np.arange(-180, 180, step=360/365.25)
    anti_SunRA_data = []
    for i in anti_lamda:
        anti_SunRA_data.append(i + 2.45 * np.sin(np.radians(2*i)))
    delta_day_data = np.arange(len(anti_SunRA_data))
    delta_day = np.interp(surveyRA, anti_SunRA_data, delta_day_data)
    anti_SunRA_datetime = VE + delta_day*u.day
    anti_SunRA_day = Time(anti_SunRA_datetime.iso.split()[0])
    return anti_SunRA_day

def find_deadtime(tile, Now, duration, subexposure, period, satellite):
    """ 
    Determines whether deadtime is required by checking if the 'tile' is visible throughout the 'subexposure' time,
    which starts 'duration' after 'Now'. Calculates the deadtime duration from 'Now' by scanning the visibility per 
    minute over an orbital 'period'.

    Parameters
    ----------
        Now : Astropy Time object
            The current time of observation.
        duration : float
            The time in seconds from 'Now' before the perceived imaging start time (assuming no deadtime).
        subexposure : float
            The continuous open-shutter time in seconds.
        period : float
            Satellite's orbital period in minutes. 
    Returns
    -------
        deadtime : float
            The time in seconds when the 'tile' is not visible (e.g. occulted by the Earth's limb).
    """
    import timeit
    code_start = timeit.default_timer()
    
    if tile.avoid_bright_objects(satellite, Now+duration*u.s) and tile.avoid_bright_objects(satellite, Now+duration*u.s+subexposure*u.s):
        # If the tile is visible throughout the open-shutter time,
        # no deadtime is required
        deadtime = 0
    else:
        # Or else determine the deadtime 
        # Scan for the tile's visibility window per minute over an orbital period
        vis = tile.get_visibility_windows(Now, Now+period*u.min, 60, satellite)
        # Deadtime is the time from 'Now' to the start of new visible cycle.
        if vis.num == 0:
            raise ValueError("The target has no visibility window currently.")
        elif vis.num == 1:
            deadtime = (vis.windows[0][0] - Now).sec
        else:
            deadtime = (vis.windows[1][0] - Now).sec
        print(f'deadtime = {round(deadtime/60, 2)} min, {round((deadtime/60)/period*100, 2)} % of the orbit')

        runtime = timeit.default_timer() - code_start
        print(f"runtime: {runtime} sec")
    return deadtime

from shapely.geometry import Polygon
""" Functions for Small Surveys """
def _find_footprint_start_time(candidate_day, vertex_targets, tile_targets, period, satellite):
    """
    Checks whether all 'vertex_targets' of the survey footprint are visible on 'candidate_day'. 
    Returns observing 'start_time' of the 'candidate_day' if so, and 'None' otherwise. 
    Helper function for find_daily_start_time().

    Parameters
    ----------
        candidate_day : Astropy Time object
        vertex_targets : list of Target objects
            A list of the four vertices of the footprint's rectangular bounds.
        tile_targets : list of Target objects
            A list of target objects converted from 'tile_centers'.
        period : float
            The satellite's orbital period in minutes. 
    Returns
    -------
        start_time : Astropy Time object, or None
    """
    coarse_start_time = candidate_day
    for vertex in vertex_targets:
        # Check if the footprint is visible throughout the day
        # Coarse calculation: step through half a day per 30 minutes
        coarse_vis = vertex.get_visibility_windows(candidate_day, candidate_day+12*u.hour, 1800, satellite)
        if coarse_vis.num == 0:
            return None
        # Pick start_time to be the latest time in the day
        # (since each tile has its first visibility at different times in the day) 
        if coarse_vis.list[0][0].datetime.hour*60 + coarse_vis.list[0][0].datetime.minute > coarse_start_time.datetime.hour*60 + coarse_start_time.datetime.minute:
            coarse_start_time = coarse_vis.windows[0][0]
    
    # Fine calculation: step through an orbital period per two minutes
    vis = tile_targets[0].get_visibility_windows(coarse_start_time, coarse_start_time+period*u.min, 2*60, satellite)
    # Define the start time to be the beginning of a new visibility cycle
    start_time = vis.get_start_time()
    return start_time

def find_daily_start_time(day_num, boundary_coords, tile_targets, period, VE, satellite, specific_day=None):
    """
    Determines the optimal start time for observing the small survey on each visible day(s), i.e. the day(s) 
    the survey is visible. For daily cadence surveys, 'day_num' specifies the number of required observation 
    days; the function will first search for all the visible days. If 'day_num' is 1, the 'specific_day' 
    parameter allows the user to specify the exact day of interest. 

    Parameters
    ----------
        day_num : int
            The number of days for which the user wants to determine the observing efficiency of the small survey.
        boundary_coords : Numpy array
            An array of footprint boundary coordinates (RA, DEC) in degrees.
        tile_targets : list of Target objects
            A list of target objects converted from 'tile_centers'.
        period : float
            The satellite's orbital period in minutes.
        VE : Astropy Time object
            Vernal Equinox of the current year in UTC.
            e.g., VE = Time("2024-3-20 3:6:0") for March 20, 2024 at 03:06 UTC   
        specific_day : Astropy Time object, optional
            If 'day_num' is 1, this parameter allows the user to specify the day to view the survey's observing efficiency.
            If left as None, it picks 'anti_SunRA_day' as default.  
    Returns
    -------
        start_times : list of Astropy Time objects
    """
    # Convert 'boundary_coords' to a list of Target objects representing the survey's rectangular bounds
    minRA, minDEC, maxRA, maxDEC = Polygon(boundary_coords).bounds
    vertex_targets = [InertialTarget(RA, DEC) for RA in [minRA, maxRA] for DEC in [minDEC, maxDEC]]
    # Define the RA of the survey center
    surveyRA = (minRA + maxRA)/2

    # Find the day when the survey is in the anti-Sun direction seen from Earth
    anti_SunRA_day = _find_anti_SunRA_day(surveyRA, VE)

    # Set the starting day
    if day_num==1 and specific_day:
        start_day = specific_day
    else:
        # Note: for cadence surveys requiring daily observations over months, 
        # start the visible day search from 'anti_SunRA_day'
        start_day = anti_SunRA_day

    start_times = []
    add = True
    add_stopped = False
    subtract_stopped = False
    i = 1

    # To search for all the visible days, start from 'anti_SunRA_day', 
    # then alternate between searching one day backward and one day forward until one end reaches a limit, 
    # then continue searching in the remaining direction until that end also reaches its limit.
    while len(start_times) < day_num:
        if not add_stopped and add:
            offset = (i // 2) if not subtract_stopped else i
            candidate_day = start_day + offset * u.day
            start_time = _find_footprint_start_time(candidate_day, vertex_targets, tile_targets, period, satellite)
            if start_time:
                print(start_time)
                start_times.append(start_time)
            else:
                add_stopped = True # Stop adding once no more valid days can be found
                add = not add # and switch to subtracting
                i = i//2
        elif not subtract_stopped and not add:
            offset = (i // 2) if not add_stopped else i
            candidate_day = start_day - offset * u.day
            start_time = _find_footprint_start_time(candidate_day, vertex_targets, tile_targets, period, satellite)
            if start_time:
                print(start_time)
                start_times.insert(0, start_time)
            else:
                subtract_stopped = True # Stop subtracting once no more valid days can be found
                add = not add # and switch to adding
                i = i//2-1
        
        # If both directions are terminated
        if add_stopped and subtract_stopped:
            break
        # Alternate between adding and subtracting when both directions are valid
        if not add_stopped and not subtract_stopped:
            add = not add  
        i += 1   
    return start_times